In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.svm import SVC

import matplotlib.pyplot as plt

In [2]:
# читаем датасет
df = pd.read_csv( 'adult.csv' )
# все признаки
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [3]:
# анализируемые признаки
used_cols = [
        'age', 
    'workclass', 
    # 'fnlwgt', # пока не трогаю, оценку ухудшает
    #  'education', # есть цифровой признак - этот не нужен - попробовал, ухудшает score
        'educational-num',
       'marital-status', 
       'occupation', 
       'relationship', # пока не трогаю
       'race', # пока не трогаю
       'gender', # 
       'capital-gain', # пока не трогаю
    #    'capital-loss', # пока не трогаю -  сильно (на 3%) ухудшает score. странно, может нужно с обратным знаком брать?
       'hours-per-week',
    #    'native-country', # интересно - ухудшает оценку
    #    'income' - целевой признак
       ]
# нечисловые признаки
cat_cols = ['gender', 'race', 'marital-status','occupation','relationship','education','workclass', 'native-country']

# TODO - использовать обучающую и тестовоую выборки
X = df[used_cols]
X = pd.get_dummies(X, columns=[c for c in cat_cols if c in used_cols])
le = LabelEncoder()
le.fit( df['income'] )
# le.classes_
y = pd.Series( data = le.transform( df['income'] ) )
model = LogisticRegression()
model.fit( X, y )
predictions = model.predict_proba( X )
model.predict(X)
model.score(X, y)

0.844682854919946

0.844682854919946 - максимальный score на 2020-12-21--21-10 - просто за счет выбора колонок


In [20]:
model2 = SVC(kernel='poly', degree=2) # TODO зависает на большой выборке - посмотреть на нормальный размер выборки
# TODO нормировать нормальные величины (и тут, и в линейной?)
model2.fit( X[:100], y[:100] )
# model2.predict_proba(X)[:5]
model2.score(X[500:1000],y[500:1000])

0.792

In [ ]:
# TODO - переписать на pipeline вроде этого отсюда https://medium.com/bigdatarepublic/integrating-pandas-and-scikit-learn-with-pipelines-f70eb6183696

transformer = Pipeline([
    ('features', FeatureUnion(n_jobs=1, transformer_list=[
        # Part 1
        ('boolean', Pipeline([
            ('selector', TypeSelector('bool')),
        ])),  # booleans close
        
        ('numericals', Pipeline([
            ('selector', TypeSelector(np.number)),
            ('scaler', StandardScaler()),
        ])),  # numericals close
        
        # Part 2
        ('categoricals', Pipeline([
            ('selector', TypeSelector('category')),
            ('labeler', StringIndexer()),
            ('encoder', OneHotEncoder(handle_unknown='ignore')),
        ]))  # categoricals close
    ])),  # features close
])  # pipeline close